In [1]:
import requests
from bs4 import BeautifulSoup
import json, pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
keyword = "옥수동"

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
url = "https://m.land.naver.com/search/result/{}".format(keyword)
res = requests.get(url, headers=headers)
res.raise_for_status()

soup = (str)(BeautifulSoup(res.text, "lxml"))

In [6]:
req_URL = "https://m.land.naver.com/complex/getComplexArticleList?hscpNo=567&tradTpCd=A1&order=point_&showR0=N&page=1"
res = requests.post(req_URL, headers=headers)
json_str = json.loads(json.dumps(res.json()))


APTlist = json_str['result']['list']
df = pd.DataFrame(APTlist)
display(df)

df.to_csv('haha.csv', encoding='utf-8-sig')

,repImgUrl,atclNo,repImgTpCd,vrfcTpCd,atclNm,bildNm,tradTpCd,tradTpNm,rletTpCd,rletTpNm,spc1,spc2,flrInfo,atclFetrDesc,cfmYmd,prcInfo,sameAddrCnt,sameAddrDirectCnt,sameAddrHash,sameAddrMaxPrc,sameAddrMinPrc,tradCmplYn,tagList,atclStatCd,cpid,cpNm,cpCnt,cpLinkVO,rltrNm,directTradYn,direction,tradePriceHan,tradeRentPrice,tradePriceInfo,tradeCheckedByOwner,dtlAddr,dtlAddrYn,repImgThumb
0,,2436001945,,OWNER,옥수현대,101동,A1,매매,A01,아파트,140.76,123.51,13/15,"확장특올수리,붙박이장풍부,바닥원목우드,정밀안전진단중.",24.07.22,21억,1,0,27A01A1N0ea72184336dc4014f74db42c1c2dc06fcf982...,21억,21억,N,"[25년이상, 올수리, 역세권, 대형평수]",R0,hkdotcom,한경부동산,1,"{'cpId': 'hkdotcom', 'mobileArticleLinkTypeCod...",현대공인중개사,N,남향,,0,,False,,N,NaN
1,,2435795379,,OWNER,옥수현대,102동,A1,매매,A01,아파트,98.91,84.9,3/15,"전세낀 재건축투자, 옥수역2분거리의 조용한 숲속아파트,",24.07.20,15억,2,0,26A01A1N9578cae16ab461a5188f80263d4ae1f16241c2...,15억,15억,N,"[25년이상, 역세권, 방세개, 화장실한개]",R0,NEONET,부동산뱅크,1,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",옥수파크부동산공인중개사사무소,N,동향,,0,,False,,N,NaN
2,,2435783820,,OWNER,옥수현대,104동,A1,매매,A01,아파트,118.44,104.38,1/15,수리잘된집 재건축정밀안전진단중 세안고갭투자,24.07.20,"15억 5,000",3,0,25A01A1N6ecc69ab4d545cbd05a284dc0afad6f4642e37...,"15억 5,000","15억 5,000",N,"[25년이상, 역세권, 1층, 방네개이상]",R0,bizmk,매경부동산,3,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",코리아공인중개사사무소,N,동향,,0,,False,,N,NaN
3,/20240720_13/land_naver_17214397897351llm4_JPE...,2435688070,10,OWNER,옥수현대,104동,A1,매매,A01,아파트,118.44,104.38,10/15,고층 남동향 재건축진행중 인근 공원숲조망 사이한강뷰,24.07.19,20억,1,0,26A01A1Nfe8f2545c45923d9551113d3b2529bc2a68972...,20억,20억,N,"[25년이상, 역세권, 방네개이상, 화장실두개]",R0,bizmk,매경부동산,1,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",단지내한강부동산공인중개사사무소,N,남동향,,0,,False,,N,f130_98
4,/20240716_85/land_naver_1721104519020h9PcY_JPE...,2434988235,10,OWNER,옥수현대,104동,A1,매매,A01,아파트,118.44,104.38,12/15,확장올수리 한강조망최고 숲조망 입주협의,24.07.16,20억,1,0,26A01A1N748b68bca4e6b98b5343a7733cf4f5aed7be9f...,20억,20억,N,"[25년이상, 올수리, 역세권, 방네개이상]",R0,NEONET,부동산뱅크,1,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",우리공인중개사사무소,N,남동향,,0,,False,,N,f130_98
5,/20240716_59/land_naver_1721104540111QoiOG_JPE...,2434988590,10,OWNER,옥수현대,106동,A1,매매,A01,아파트,98.91,84.9,1/15,"옥수초역세권 재건축추진중 방3,화2 올확장특올수리 입주협의",24.07.16,15억,3,0,25A01A1Ndf5c3b64e7e673e8caf6868dd727f40f3c9517...,15억,15억,N,"[25년이상, 올수리, 역세권, 1층]",R0,bizmk,매경부동산,3,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",코리아공인중개사사무소,N,남향,,0,,False,,N,f130_98
6,,2435708487,,OWNER,옥수현대,107동,A1,매매,A01,아파트,92.57,71.05,고/12,확장 올수리 남향 방3개 단지상가앞 재건축 추진중,24.07.20,14억,1,0,27A01A1Na0ab16dce41ee8b4a9849273e3ddbf7b3e67e4...,14억,14억,N,"[25년이상, 올수리, 역세권, 방세개]",R0,NEONET,부동산뱅크,1,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",우리공인중개사사무소,N,남향,,0,,False,,N,NaN
7,,2435419674,,OWNER,옥수현대,106동,A1,매매,A01,아파트,92.57,71.05,고/15,입주협의 l 옥수역초역세권 l 기본깔끔 l 로얄동 l 급매,24.07.18,"13억 5,000",2,0,26A01A1Nae4c8cf770f4ea331c770bac8bc0788303fb5d...,"13억 5,000","13억 5,000",N,"[25년이상, 역세권, 급매, 방세개]",R0,bizmk,매경부동산,2,"{'cpId': 'bizmk', 'mobileArticleLinkTypeCode':...",단지내행운공인중개사사무소,N,남동향,,0,,False,,N,NaN
8,/20240717_267/land_naver_1721196441725lzGfw_JP...,2435204301,10,OWNER,옥수현대,101동,A1,매매,A01,아파트,140.76,123.51,고/15,"42타입,공동 환영,로얄동 로얄층,샷시까지 올수리,최고의 매물",24.07.17,20억,4,0,27A01A1N935a10b36065e95db1d8aa1370e7f4c94803ab...,21억,20억,N,"[25년이상, 올수리, 역세권, 대형평수]",R0,NEONET,부동산뱅크,4,"{'cpId': 'NEONET', 'mobileArticleLinkTypeCode'...",삼성리버젠 공인중개사 사무소,N,남향,,0,,False,,N,f130_98
9,,2435337720,,NDOC1,옥수현대,107동,A1,매매,A01,아파트,76.97,58.72,중/12,"주인직접,수리예쁜집,신혼부부추천,조용한동,재건축추진중",24.07.18,"12억 7,000",2,0,26A01A1Ncaf212aecf8a94b1d2ec4d579f8bd21bed6f07...,"12억 7,000","12억 7,000",N,"[25년이상, 역세권, 방두개, 화장실한개]",R0,asil,아실,2,"{'cpId': 'asil', 'mobileArticleUrl': 'https://...",반포서울공인중개사사무소,N,남향,,0,,False,,N,NaN


PermissionError: [Errno 13] Permission denied: 'haha.csv'

In [10]:
URL = "https://m.land.naver.com/complex/getComplexArticleList"
param = {
    'hscpNo': '567',
    'tradTpCd': 'B1',
    'order': 'point',
    'showR0': 'N',
    'page': '1',

}
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
    'Referer': 'https://m.land.naver.com/'
}

req_URL = "https://m.land.naver.com/complex/getComplexArticleList"

resp = requests.post(req_URL, params=param, headers=header)
json_str = json.loads(json.dumps(resp.json()))

APTlist = json_str['result']['list']
df = pd.DataFrame(APTlist)
display(df)

page = 1
while True:
    page += 1
    param['page'] = page

    req_URL = "https://m.land.naver.com/complex/getComplexArticleList"

    resp = requests.post(req_URL, params=param, headers=header)
    json_str = json.loads(json.dumps(resp.json()))


    if len(json_str['result']['list']) == 0:
         break

    APTlist = json_str['result']['list']
    nex_df = pd.DataFrame(APTlist)
    df = pd.concat([df,nex_df])

display(df)

""


""
